In [127]:
import importlib
import functions

importlib.reload(functions) 

from functions import *

In [2]:
BASE_LINK = 'https://data.who.int/countries'

In [3]:
driver = start_browser()

There was an error managing msedgedriver (error sending request for url (https://msedgedriver.azureedge.net/LATEST_RELEASE_126_LINUX)); using driver found in the cache


In [100]:
open_page(driver, BASE_LINK)

In [5]:
counteries = get_list_of_countries(driver)

In [6]:
print(f"Total {len(counteries)} countries found")

Total 194 countries found


In [101]:
Data = []

In [ ]:
def mapper():
    '''
    Takes a dictionary and returns a dictonary of values
    calcualte the values for missing years 
    Approch is to calculate the slope between very two points then then pridict the in between values
    improve this approach using GPT
    Dictonary must contains 24 data points
    '''
    pass

In [102]:
for country in counteries:
    open_page(driver, country)
    
    name = get_country_name(driver)
    expenditure = get_current_health_expenditure(driver).split('(')[0]
    region_name = get_who_region(driver)
    income_level = get_world_bank_income_level(driver)
    growth_rate = get_population_growth_rate(driver)
    data_list = get_population_data(driver)
    
    

    # life_expentancy = get_life_expentency_data(driver)
    # life_expentancy = mapper(life_expentancy)
    
    # code to click the health expenditure button 

    # health_expenditure = get_health_life_expectancy_data(driver)
    # health_expenditure = mapper(health_expenditure)

    # group_A_links = get_group_A_indicators(driver)
    # group_B_links = get_group_B_indicators(driver)

    # Now fetch data for each indicator
    # indicators = {}
    # for i in group_A_links:
        # ind_data = get_indicator_data_for_group_A(i.value)
        # indicators[i.key] = ind_data
    # for i in group_B_links:
        # ind_data = get_indicator_data_for_group_B(i.value)
        # indicators[i.key] = ind_data
    
    # Now we have scraped all data from current page. Now change the pages
    for i in data_list[:25]:
        dictonary = {
            'Country': name,
            'Current Health Expenditure': expenditure,
            'WHO Region': region_name,
            'World Bank Income Level': income_level,
            'Population Growth Rate': growth_rate,
            'Population': i[1],
            'Year': i[0],
        }
        
        Data.append(dictonary)

    break 


In [97]:
get_life_expentency_data(driver)


Getting life expectancy data...
_______________________________
None Female
2000 55.4
2010 60.3
2015 62.3
2019 63.2
2021 61
_______________________________
None Male
2000 54.6
2010 59.6
2015 61
2019 63.3
2021 57.4
_______________________________
None Total
2000 55
2010 59.9
2015 61.7
2019 63.2
2021 59.1
_______________________________


In [128]:

url_A = "https://data.who.int/indicators/i/49AC786/77D059C?m49=004"
group_A_data = get_indicator_data_for_group_A(driver, url_A)
print(group_A_data)


InvalidArgumentException: Message: invalid argument
  (Session info: MicrosoftEdge=126.0.2592.102)
Stacktrace:
#0 0x56169a9c467e <unknown>
#1 0x56169a681545 <unknown>
#2 0x56169a66af15 <unknown>
#3 0x56169a669a3a <unknown>
#4 0x56169a669b97 <unknown>
#5 0x56169a684309 <unknown>
#6 0x56169a70dc5e <unknown>
#7 0x56169a6ef972 <unknown>
#8 0x56169a70d58e <unknown>
#9 0x56169a6ef713 <unknown>
#10 0x56169a6c0a43 <unknown>
#11 0x56169a6c17fe <unknown>
#12 0x56169a994151 <unknown>
#13 0x56169a9972ad <unknown>
#14 0x56169a996d32 <unknown>
#15 0x56169a997735 <unknown>
#16 0x56169a9854db <unknown>
#17 0x56169a997ad1 <unknown>
#18 0x56169a971e05 <unknown>
#19 0x56169a9b55b8 <unknown>
#20 0x56169a9b5749 <unknown>
#21 0x56169a9c39f1 <unknown>
#22 0x7fd9b674bdab <unknown>


In [80]:

# Get data for a specific group B indicator
url_B = "https://data.who.int/indicators/i/CCCEBB2/217795A?m49=004"
group_B_data = get_indicator_data_for_group_B(driver, url_B)
print(group_B_data)

[]


## NAJAM

**Things to do**
- 1 Name of country
- 2 Current health expenditure (% of GDP)
- 3 WHO region
- 4 World Bank income level
- 5 Population growth rate
- 6 Leading cause of death
- 7 Leading cause of under 5 mortality
- 8 extract data from population graph from 2000 to 2024 (get twenty rows w.r.t two columns year and population)
- 9 extract data from life expentency at birth graph (get twenty data points from 2000 to 2024 with two columns year and total (not male or female))
- 10 extract data from Healthy life expectancy (HALE) at birth graph (get twenty data points from 2000 to 2024 with two columns year and total (not male or female))


## SHAHZAD

**Things to do**


- 3 two functions to extract **visit indicator** links for following categories (divide them in group A and B)
   - 3.1 Number of new HIV infections
   - 3.2 People living with tuberculosis (TB)
   - 3.3 Malaria cases
   - 3.4 Probability of dying from non-communicable diseases
   - 3.5 Road traffic deaths
   - 3.6 Suicide deaths
   - 3.7 Prevalence of hypertension
   - 3.8 Adult obesity
   - 3.9 Tobacco use
   - 3.10 Alcohol consumption
   - 3.11 Safely managed sanitation
   - 3.12 Fine particulate matter
   - 3.13 UHC index score
   - 3.14 Births attended by skilled health personnel
   - 3.15 Family planning
   - 3.16 DTP3 immunization
   - 3.17 MCV2 immunization
   - 3.18 Interventions against NTDs
   - 3.19 Density of doctors
   - 3.20 Density of nurses
   - 3.21 Density of pharmacists
   - 3.22 Density of dentists
   - 3.23 WASH development assistance
   - 3.24 Financial hardship
- 4 A general function to extract visit indicators of this kind of pages (https://data.who.int/indicators/i/49AC786/77D059C?m49=004)
   - 4.1 return 24 rows with two columns year and count.
- 5 a general function to extract the other kind of function
   - 5.1 get 23-24 years data from line graph displayed on indicator page (example https://data.who.int/indicators/i/CCCEBB2/217795A?m49=004)